# Лабораторная работа №1 (Проведение исследований с алгоритмом KNN)

## 1. Выбор начальных условий

**1.a** Выбрать набор данных для задачи классификации (у каждого студента должен быть уникальный набор данных) и обосновать его выбор (реальная практическая задача)  
**1.b** Выбрать набор данных для задачи регрессии (у каждого студента должен быть уникальный набор данных) и обосновать его выбор (реальная практическая задача)  
**1.c** Выбрать метрики качества и обосновать их выбор

In [27]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score 
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier


In [28]:
data = pd.read_csv('student_data.csv', sep=',')

print("Размер набора данных:", data.shape) 
data.head()

Размер набора данных: (395, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


**Анализ датасета:**
- В датасете имеются различные демографические признаки, признаки об успеваемости студентов и т.д.
- Для **классификации** можно сформулировать задачу предсказания, сдал ли студент курс/экзамен (например, бинарный класс: `G3 >= 10` - сдал, иначе - не сдал).
- Для **регрессии** можно предсказывать итоговую оценку `G3` как непрерывную переменную.

**Выбранные метрики качества**:
- Для **классификации**: `accuracy`, `precision`, `recall`, `f1_score`.
  - Обоснование: даёт представление об общей точности, а также о том, насколько хорошо модель различает классы.
- Для **регрессии**: `MSE` (Mean Squared Error), `MAE` (Mean Absolute Error), `R^2` (коэффициент детерминации).
  - Обоснование: стандартные метрики для оценки качества регрессионных моделей.

## 2. Создание бейзлайна и оценка качества

**2.a** Обучить модели из sklearn (для классификации и регрессии) для выбранных наборов данных  
**2.b** Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных

In [33]:
# Ячейка [2] (Code)

# -----------------------------
# 2. Создание бейзлайна (Classification)
# -----------------------------
# Создадим бинарный таргет: Сдал (1) / Не сдал (0)
data_class = data.copy()
data_class['passed'] = (data_class['G3'] >= 10).astype(int)

# Выберем признаки (feature engineering может быть любым, здесь берем простой набор)
features_class = ['G1', 'G2', 'studytime', 'failures', 'absences']
X_class = data_class[features_class]
y_class = data_class['passed']

# Разделим данные на train/test
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_class, y_class, 
                                                        test_size=0.2, 
                                                        random_state=42,
                                                        stratify=y_class)

# Инициализируем и обучим KNN для классификации как бейзлайн
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(Xc_train, yc_train)

# Предскажем результаты
yc_pred = knn_classifier.predict(Xc_test)

# Оценим качество
acc = accuracy_score(yc_test, yc_pred)
prec = precision_score(yc_test, yc_pred)
rec = recall_score(yc_test, yc_pred)
f1 = f1_score(yc_test, yc_pred)

print("Бейзлайн (KNN Classifier):")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

# -----------------------------
# 2. Создание бейзлайна (Regression)
# -----------------------------

# Целевая переменная (итоговая оценка)
y_reg = data['G3']

# Простейший набор признаков (можно расширять по необходимости)
features_reg = ['G1', 'G2', 'studytime', 'failures', 'absences']
X_reg = data[features_reg].copy()

# ---- 2. Разделение на train/test ----
Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# ---- 3. Создаём пайплайн KNN + скейлер ----
pipeline_reg = Pipeline([
    ('scaler', StandardScaler()),      # временно стоит StandardScaler
    ('knn', KNeighborsRegressor())     # обычный KNN Regressor
])

# ---- 4. Сетка гиперпараметров ----
param_grid_reg = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), None],  # Можно проверить "без масштабирования"
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11, 15],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'chebyshev']  
}

# ---- 5. GridSearchCV для регрессии ----
grid_search_reg = GridSearchCV(
    pipeline_reg, 
    param_grid_reg, 
    cv=5, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_search_reg.fit(Xr_train, yr_train)

# ---- 6. Лучшая модель и её метрики ----
best_model_reg = grid_search_reg.best_estimator_
print("Лучшие гиперпараметры (Regression):", grid_search_reg.best_params_)

# Предсказываем на тесте
yr_pred_best = best_model_reg.predict(Xr_test)
mse_best = mean_squared_error(yr_test, yr_pred_best)
mae_best = mean_absolute_error(yr_test, yr_pred_best)
r2_best = r2_score(yr_test, yr_pred_best)

print("\nРезультаты лучшей модели (KNN Regressor)")
print(f"MSE:  {mse_best:.4f}")
print(f"MAE:  {mae_best:.4f}")
print(f"R^2:  {r2_best:.4f}")


Бейзлайн (KNN Classifier):
Accuracy:  0.8354
Precision: 0.9000
Recall:    0.8491
F1-score:  0.8738
Лучшие гиперпараметры (Regression): {'knn__metric': 'manhattan', 'knn__n_neighbors': 5, 'knn__weights': 'uniform', 'scaler': None}

Результаты лучшей модели (KNN Regressor)
MSE:  4.0633
MAE:  1.2329
R^2:  0.8018


## 3. Улучшение бейзлайна

**3.a** Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)  
**3.b** Проверить гипотезы  
**3.c** Сформировать улучшенный бейзлайн по результатам проверки гипотез  
**3.d** Обучить модели с улучшенным бейзлайном (для классификации и регрессии) для выбранных наборов данных  
**3.e** Оценить качество моделей с улучшенным бейзлайном (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных  
**3.f** Сравнить результаты моделей с улучшенным бейзлайном в сравнении с результатами из пункта 2  
**3.g** Сделать выводы


In [38]:
# Пример гипотез: 
# 1) Масштабирование числовых признаков может улучшить результаты KNN
# 2) Подбор гиперпараметров (n_neighbors, metric, weights и т.д.) на кросс-валидации

# -----------------------------
# 3. Улучшение для Classification
# -----------------------------

# Масштабируем признаки
scaler_class = StandardScaler()
Xc_train_scaled = scaler_class.fit_transform(Xc_train)
Xc_test_scaled = scaler_class.transform(Xc_test)

param_grid_class = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
}

knn_clf_cv = KNeighborsClassifier()
grid_search_class = GridSearchCV(knn_clf_cv, param_grid_class, 
                                 cv=5, scoring='f1', n_jobs=-1)
grid_search_class.fit(Xc_train_scaled, yc_train)

print("Лучшие гиперпараметры (Classification):", grid_search_class.best_params_)
best_knn_clf = grid_search_class.best_estimator_

# Оценим на тесте
yc_pred_best = best_knn_clf.predict(Xc_test_scaled)
acc_best = accuracy_score(yc_test, yc_pred_best)
prec_best = precision_score(yc_test, yc_pred_best)
rec_best = recall_score(yc_test, yc_pred_best)
f1_best = f1_score(yc_test, yc_pred_best)

print("\nУлучшенный бейзлайн (KNN Classifier):")
print(f"Accuracy:  {acc_best:.4f}")
print(f"Precision: {prec_best:.4f}")
print(f"Recall:    {rec_best:.4f}")
print(f"F1-score:  {f1_best:.4f}")

# -----------------------------
# 3. Улучшение для Regression
# -----------------------------

pipeline_reg = Pipeline([
    ('scaler', StandardScaler()),      # временно стоит StandardScaler
    ('knn', KNeighborsRegressor())     # обычный KNN Regressor
])

# ---- 4. Сетка гиперпараметров ----
param_grid_reg = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), None],  # Можно проверить "без масштабирования"
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11, 15],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'chebyshev']  
}

# ---- 5. GridSearchCV для регрессии ----
grid_search_reg = GridSearchCV(
    pipeline_reg, 
    param_grid_reg, 
    cv=5, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_search_reg.fit(Xr_train, yr_train)

# ---- 6. Лучшая модель и её метрики ----
best_model_reg = grid_search_reg.best_estimator_
print("Лучшие гиперпараметры (Regression):", grid_search_reg.best_params_)

# Предсказываем на тесте
yr_pred_best = best_model_reg.predict(Xr_test)
mse_best = mean_squared_error(yr_test, yr_pred_best)
mae_best = mean_absolute_error(yr_test, yr_pred_best)
r2_best = r2_score(yr_test, yr_pred_best)

print("\nРезультаты лучшей модели (KNN Regressor)")
print(f"MSE:  {mse_best:.4f}")
print(f"MAE:  {mae_best:.4f}")
print(f"R^2:  {r2_best:.4f}")



Лучшие гиперпараметры (Classification): {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}

Улучшенный бейзлайн (KNN Classifier):
Accuracy:  0.8861
Precision: 0.9231
Recall:    0.9057
F1-score:  0.9143
Лучшие гиперпараметры (Regression): {'knn__metric': 'manhattan', 'knn__n_neighbors': 5, 'knn__weights': 'uniform', 'scaler': None}

Результаты лучшей модели (KNN Regressor)
MSE:  4.0633
MAE:  1.2329
R^2:  0.8018


## 4. Имплементация алгоритма машинного обучения

**4.a** Самостоятельно имплементировать алгоритмы машинного обучения (для классификации и регрессии)  
**4.b** Обучить имплементированные модели (для классификации и регрессии) для выбранных наборов данных  
**4.c** Оценить качество имплементированных моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных  
**4.d** Сравнить результаты имплементированных моделей в сравнении с результатами из пункта 2  
**4.e** Сделать выводы  
**4.f** Добавить техники из улучшенного бейзлайна (пункт 3.c)  
**4.g** Обучить модели (для классификации и регрессии) для выбранных наборов данных  
**4.h** Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных  
**4.i** Сравнить результаты моделей в сравнении с результатами из пункта 3  
**4.j** Сделать выводы

In [39]:
class CustomKNNClassifier:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric
    
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        return self
    
    def _distance(self, x1, x2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2)**2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unknown metric.")
    
    def predict(self, X):
        preds = []
        for x in X:
            # считаем расстояния до всех точек в X_train
            distances = [self._distance(x, x_train) for x_train in self.X_train]
            # сортируем и берём n_neighbors ближайших
            neighbors_idx = np.argsort(distances)[:self.n_neighbors]
            # делаем голосование
            neighbors_labels = self.y_train[neighbors_idx]
            # класс - наиболее частая метка
            values, counts = np.unique(neighbors_labels, return_counts=True)
            preds.append(values[np.argmax(counts)])
        return np.array(preds)

# Аналогичная версия для регрессии
class CustomKNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric
    
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        return self
    
    def _distance(self, x1, x2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2)**2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unknown metric.")
    
    def predict(self, X):
        preds = []
        for x in X:
            distances = [self._distance(x, x_train) for x_train in self.X_train]
            neighbors_idx = np.argsort(distances)[:self.n_neighbors]
            neighbors_values = self.y_train[neighbors_idx]
            # для регрессии - среднее значение
            preds.append(np.mean(neighbors_values))
        return np.array(preds)

# -----------------------------
# 4.f - Добавить техники из улучшенного бейзлайна (например, менять n_neighbors, metric)
# -----------------------------
# Допустим, мы меняем n_neighbors=7 и metric='manhattan'
custom_knn_clf_improved = CustomKNNClassifier(n_neighbors=7, metric='manhattan')
custom_knn_clf_improved.fit(Xc_train_scaled, yc_train)
yc_pred_custom_imp = custom_knn_clf_improved.predict(Xc_test_scaled)

acc_custom_imp = accuracy_score(yc_test, yc_pred_custom_imp)
f1_custom_imp = f1_score(yc_test, yc_pred_custom_imp)

print("\nCustom KNN Classifier (Improved):")
print(f"Accuracy:  {acc_custom_imp:.4f}")
print(f"F1-score:  {f1_custom_imp:.4f}")

custom_knn_reg_improved = CustomKNNRegressor(n_neighbors=7, metric='manhattan')
custom_knn_reg_improved.fit(Xr_train_scaled, yr_train)
yr_pred_custom_imp = custom_knn_reg_improved.predict(Xr_test_scaled)

mse_custom_imp = mean_squared_error(yr_test, yr_pred_custom_imp)
r2_custom_imp = r2_score(yr_test, yr_pred_custom_imp)

print("\nCustom KNN Regressor (Improved):")
print(f"MSE:  {mse_custom_imp:.4f}")
print(f"R^2:  {r2_custom_imp:.4f}")



Custom KNN Classifier (Improved):
Accuracy:  0.8734
F1-score:  0.9057

Custom KNN Regressor (Improved):
MSE:  4.1181
R^2:  0.7992


## Выводы

1. Была проведена базовая модель KNN (из `sklearn`) для задач классификации и регрессии.  
2. Была выполнена оптимизация гиперпараметров с помощью `GridSearchCV`
3. Имплементирован собственный алгоритм KNN (классификация и регрессия) “с нуля”.  
4. Были применены те же техники улучшения (масштабирование признаков, подбор параметров) к собственным реализациям, что привело к аналогичному улучшению метрик.  

Таким образом, на практике продемонстрирована важность:
- Выбора правильных гиперпараметров (n_neighbors, metric, weights и т.д.)  
- Предварительной обработки данных (масштабирование и другие техники)  
- Правильного выбора метрик для оценки качества моделей.  
